In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

# Specify the file path of the Excel file
file_path = '/kaggle/input/d2cohennew3/Cohennew3binary.xlsx'

# Read the 'main' sheet of the Excel file into a DataFrame
df2 = pd.read_excel(file_path, sheet_name='section info')

# Now you can work with the DataFrame 'df' containing the data from the 'main' sheet
# For example, you can print the first few rows of the DataFrame
df2.head()

In [ ]:
df2['Section'] = df['Section'].str.replace(r'(?i)\bMethod\b', 'method', regex=True)
df2['Section'] = df['Section'].str.replace(r'(?i)\bDISCUSSION\b', 'discussion', regex=True)
df2['Section'] = df['Section'].replace(r'\bResult\b', 'result', regex=True)

In [ ]:
unique_sections = df2['Section'].unique()
print(unique_sections)

In [ ]:

# Calculate the total number of unique combinations of "cited id" and "citing id"
unique_combinations = df2.groupby(["Cited Id", "Citing Id"]).size().reset_index(name="Total Combinations")

df2 = pd.merge(df2, unique_combinations, on=["Cited Id", "Citing Id"], how="left")


In [ ]:
# Count unique combinations of Cited ID and Citing ID
unique_combinations = df2.groupby(['Cited Id', 'Citing Id']).size().reset_index(name='Count')

# Calculate the total number of times each Cited ID is cited by any Citing ID
total_citations_per_cited_id = unique_combinations.groupby('Cited Id')['Count'].sum()



In [ ]:
df2 = df2.merge(total_citations_per_cited_id, on='Cited Id', how='left')

In [ ]:
df2.head()

In [ ]:
df = df.drop(columns="Count_y")

In [ ]:
# Count unique combinations of Cited ID, Citing ID, and Section
unique_combinations = df2.groupby(['Cited Id', 'Citing Id', 'Section']).size().reset_index(name='Count')

# Calculate the total number of sections in which each Citing ID cited the Cited ID
total_sections_per_citing_id = unique_combinations.groupby('Citing Id')['Section'].nunique().reset_index()

# Rename the column to 'Total Sections per Citing ID'
total_sections_per_citing_id = total_sections_per_citing_id.rename(columns={'Section': 'Total Sections per Citing ID'})

# Merge the total_sections_per_citing_id column with the original dataset
df2 = df2.merge(total_sections_per_citing_id, on='Citing Id', how='left')

In [ ]:
df2.head()

In [ ]:
# Count unique combinations of Cited ID, Citing ID, and Section
unique_combinations1 = df2.groupby(['Cited Id', 'Citing Id', 'Section']).size().reset_index(name='Count')

# Calculate the total number of citations in each section for each Citing ID
total_citations_per_section1 = unique_combinations1.groupby(['Citing Id', 'Section'])['Count'].sum().reset_index()

# Pivot the table to have sections as columns
section_pivot = total_citations_per_section1.pivot(index='Citing Id', columns='Section', values='Count')

# Fill missing values with 0 for sections with no citations
section_pivot = section_pivot.fillna(0)

# Rename the columns to 'Citation in Section'
section_pivot.columns = [f'Citation in {section}' for section in section_pivot.columns]

# Merge the section_pivot with the original dataset
df2 = df2.merge(section_pivot, on='Citing Id', how='left')

In [ ]:
df2.head()

In [ ]:
df2.to_csv('updated_data3.csv', index=False)

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
download_file('/kaggle/working/updated_data3.csv', 'out')